#Chapter 9

before we used any non alphanumeric character as the spliter for the tokeniser

this has a problem when working with urls, money, dates, emailaddress etc...

words like we're 

City names New York

other words which are commanly found together rock 'n' roll

Some Languages dont have spaces between words

So how do you find the boundaries should be? build a list of unique words from the Corpus

## Byte Pair Encoding 

build a list ov vocab using all character
then loop over each word building substrings that commonly appear in the words and add to vocab
keep doing this until max number of merges appear

Frequent words
subwords




In [1]:
import numpy as np
import re
from operator import itemgetter

Corpus = {
    'l o w _':5,
    'l o w e r _':2,
    'n e w e s t _':6,
    'w i d e s t _':3,
    'h a p p i e r _':2
}

def getPairCounts(Corpus):
    pairs = {}
    for word,fr in Corpus.items():
        symbols = word.split(' ')
        for i in range(len(symbols)-1):
            pair = (symbols[i],symbols[i+1])
            cfr = pairs.get(pair,0)
            pairs[pair] = cfr+fr
    return pairs

pairsCounts = getPairCounts(Corpus)
pairsCounts

{('l', 'o'): 7,
 ('o', 'w'): 7,
 ('w', '_'): 5,
 ('w', 'e'): 8,
 ('e', 'r'): 4,
 ('r', '_'): 4,
 ('n', 'e'): 6,
 ('e', 'w'): 6,
 ('e', 's'): 9,
 ('s', 't'): 9,
 ('t', '_'): 9,
 ('w', 'i'): 3,
 ('i', 'd'): 3,
 ('d', 'e'): 3,
 ('h', 'a'): 2,
 ('a', 'p'): 2,
 ('p', 'p'): 2,
 ('p', 'i'): 2,
 ('i', 'e'): 2}

In [3]:
def getBestPair(pairsCounts):
    return max(pairsCounts,key=pairsCounts.get)


print(getBestPair(pairsCounts))

def mergeInCorpus(bestPair,Corpus):
    newCorpus = {}
    for word in Corpus:
        newWord = re.sub(' '.join(bestPair),''.join(bestPair),word)
        newCorpus[newWord] = Corpus[word]
    return newCorpus


bestPair = getBestPair(pairsCounts)
newCorpus = mergeInCorpus(bestPair,Corpus)

newCorpus

('e', 's')


{'l o w _': 5,
 'l o w e r _': 2,
 'n e w es t _': 6,
 'w i d es t _': 3,
 'h a p p i e r _': 2}

In [6]:
def runBPE(Corpus,k):
    bpeStats = {}
    for i in range(k):
        pairsCounts = getPairCounts(Corpus)
        if not pairsCounts:
            break
        bestPair = getBestPair(pairsCounts)
        bpeStats[bestPair] = i
        Corpus = mergeInCorpus(bestPair,Corpus)
    return Corpus,bpeStats

Corpus = {
    'l o w _':5,
    'l o w e r _':2,
    'n e w e s t _':6,
    'w i d e s t _':3,
    'h a p p i e r _':2
}

newCorpus,bpeStats = runBPE(Corpus,10)


print(newCorpus)

print(bpeStats)

{'low_': 5, 'low er _': 2, 'newest_': 6, 'w i d est_': 3, 'h a p p i er _': 2}
{('e', 's'): 0, ('es', 't'): 1, ('est', '_'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'w'): 6, ('new', 'est_'): 7, ('low', '_'): 8, ('e', 'r'): 9}


In [7]:
newWord = 'lowest'
newWord2 = ' '.join(list(newWord))+' _'


def getAllPairs(word):
    pairs = []
    word = word.split(' ')
    prevChar = word[0]
    for char in word[1:]:
        pairs.append((prevChar,char))
        prevChar = char
    return pairs

pairs = getAllPairs(newWord2)

pairs

[('l', 'o'), ('o', 'w'), ('w', 'e'), ('e', 's'), ('s', 't'), ('t', '_')]

In [8]:

def getPairToBeMerged(bpeStats,pairs):
    #bpeCodes = [(pair,bpeStats[pair]) for pair in pairs if pair in bpeStats]
    bpeCodes = []
    for pair in pairs:
        if pair in bpeStats:
            bpeCodes.append((pair,bpeStats[pair]))
    if len(bpeCodes) == 0:
        return (-1,-1)
    pairToBeMerged = min(bpeCodes,key=itemgetter(1))[0]
    return pairToBeMerged

pairToBeMerged = getPairToBeMerged(bpeStats,pairs)

def mergeLetters(word,pairToBeMerged):
    newWord = re.sub(' '.join(pairToBeMerged),''.join(pairToBeMerged),word)
    return newWord


print(mergeLetters(newWord2,pairToBeMerged))

l o w es t _


In [10]:
def bpeTokenize(word,bpeStats):
    if len(word) == 1:
        return word
    word = ' '.join(list(word))+' _'
    while True:
        pairs = getAllPairs(word)
        pairToBeMerged = getPairToBeMerged(bpeStats,pairs)
        if pairToBeMerged[0] == -1:
            break
        word = mergeLetters(word,pairToBeMerged)
    return word

    
newWord = bpeTokenize('lowest',bpeStats)
print(newWord)

bpeStats

low est_


{('e', 's'): 0,
 ('es', 't'): 1,
 ('est', '_'): 2,
 ('l', 'o'): 3,
 ('lo', 'w'): 4,
 ('n', 'e'): 5,
 ('ne', 'w'): 6,
 ('new', 'est_'): 7,
 ('low', '_'): 8,
 ('e', 'r'): 9}